# Smith-Waterman local alignment implementation
Adapted application of Smith-Watermann algorithm. We apply this algorithm to genes of their genomes.

<i>Source: https://github.com/slavianap/Smith-Waterman-Algorithm#readme

In [1]:
# -*- coding: utf-8 -*-

"""

LIFE97011 - Computing
Python Programming - Assessed Exercise No. 3
Task: Smith Waterman local alignment
@Author: Slaviana Pavlovich

"""

# Importing Python packages
from enum import IntEnum
import numpy as np

# Assigning the constants for the scores
class Score(IntEnum):
    MATCH = 1
    MISMATCH = -1
    GAP = -1
    INV = 0

# Assigning the constant values for the traceback
class Trace(IntEnum):
    STOP = 0
    LEFT = 1 
    UP = 2
    DIAGONAL = 3

# Reading the fasta file and keeping the formatted sequence's name and sequence
'''
def fasta_reader(sequence_file):
    lines = open(sequence_file).readlines()
    sequence_name_row = lines[0][1:]
    sequence = lines[1]
    return sequence_name_row.replace(" ", "").strip(), sequence.strip()
'''

# Implementing the Smith Waterman local alignment
def smith_waterman(seq1, seq2):

    row = len(seq1) + 1
    col = len(seq2) + 1
    matrix = np.zeros(shape=(row, col), dtype= int)  
    tracing_matrix = np.zeros(shape=(row, col), dtype= int)  

    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)

    # Calculating the scores for all cells in the matrix

    for i in range(1, row):
        for j in range(1, col):
            # Calculating the diagonal score (match score)
            match_value = Score.MATCH if seq1.iloc[i - 1, 1] == seq2.iloc[j - 1, 1] else Score.INV if seq1.iloc[i - 1, 2] == seq2.iloc[j - 1, 2] else Score.MISMATCH
            diagonal_score = matrix[i - 1, j - 1] + match_value
            
            # Calculating the vertical gap score
            vertical_score = matrix[i - 1, j] + Score.GAP
            
            # Calculating the horizontal gap score
            horizontal_score = matrix[i, j - 1] + Score.GAP
            
            # Taking the highest score 
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)
            
            # Tracking where the cell's value is coming from    
            if matrix[i, j] == 0: 
                tracing_matrix[i, j] = Trace.STOP
                
            elif matrix[i, j] == horizontal_score: 
                tracing_matrix[i, j] = Trace.LEFT
                
            elif matrix[i, j] == vertical_score: 
                tracing_matrix[i, j] = Trace.UP
                
            elif matrix[i, j] == diagonal_score: 
                tracing_matrix[i, j] = Trace.DIAGONAL 
                
            # Tracking the cell with the maximum score
            # If we want different strings, here we can define a threshold and keep track of all the higher values
            if matrix[i, j] >= max_score:
                max_index = (i,j)
                max_score = matrix[i, j]


    # Initialising the variables for tracing
    aligned_seq1 = [ ] #""
    aligned_seq2 = [ ] #""  
    index_seq1 = [ ]
    index_seq2 = [ ]
    (max_i, max_j) = max_index

    # Tracing and computing the pathway with the local alignment
    while tracing_matrix[max_i, max_j] != Trace.STOP:
        if tracing_matrix[max_i, max_j] == Trace.DIAGONAL:
            aligned_seq1.append(seq1.iloc[max_i - 1, 2])
            aligned_seq2.append(seq2.iloc[max_j - 1, 2])

            index_seq1.append(seq1.iloc[max_i - 1, 0])
            index_seq2.append(seq2.iloc[max_j - 1, 0])

            max_i = max_i - 1
            max_j = max_j - 1
            

        elif tracing_matrix[max_i, max_j] == Trace.UP:
            aligned_seq1.append(seq1.iloc[max_i - 1, 2])
            aligned_seq2.append('-')

            index_seq1.append(seq1.iloc[max_i - 1, 0])
            index_seq2.append('-')

            max_i = max_i - 1    
            

        elif tracing_matrix[max_i, max_j] == Trace.LEFT:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2.iloc[max_j - 1, 2])

            index_seq1.append('-')
            index_seq2.append(seq2.iloc[max_j - 1, 0])
            
            max_j = max_j - 1

    return aligned_seq1, aligned_seq2, index_seq1, index_seq2, max_score
